In [2]:
import cornac
ratings = cornac.datasets.movielens.load_feedback()


Data from http://files.grouplens.org/datasets/movielens/ml-100k/u.data
will be cached into C:\Users\Administrator\.cornac\ml-100k/u.data


1.99MB [03:00, 11.0kB/s]                            

File cached!


In [3]:
ratings

[('196', '242', 3.0),
 ('186', '302', 3.0),
 ('22', '377', 1.0),
 ('244', '51', 2.0),
 ('166', '346', 1.0),
 ('298', '474', 4.0),
 ('115', '265', 2.0),
 ('253', '465', 5.0),
 ('305', '451', 3.0),
 ('6', '86', 3.0),
 ('62', '257', 2.0),
 ('286', '1014', 5.0),
 ('200', '222', 5.0),
 ('210', '40', 3.0),
 ('224', '29', 3.0),
 ('303', '785', 3.0),
 ('122', '387', 5.0),
 ('194', '274', 2.0),
 ('291', '1042', 4.0),
 ('234', '1184', 2.0),
 ('119', '392', 4.0),
 ('167', '486', 4.0),
 ('299', '144', 4.0),
 ('291', '118', 2.0),
 ('308', '1', 4.0),
 ('95', '546', 2.0),
 ('38', '95', 5.0),
 ('102', '768', 2.0),
 ('63', '277', 4.0),
 ('160', '234', 5.0),
 ('50', '246', 3.0),
 ('301', '98', 4.0),
 ('225', '193', 4.0),
 ('290', '88', 4.0),
 ('97', '194', 3.0),
 ('157', '274', 4.0),
 ('181', '1081', 1.0),
 ('278', '603', 5.0),
 ('276', '796', 1.0),
 ('7', '32', 4.0),
 ('10', '16', 4.0),
 ('284', '304', 4.0),
 ('201', '979', 2.0),
 ('276', '564', 3.0),
 ('287', '327', 5.0),
 ('246', '201', 5.0),
 ('242'

In [9]:
import pandas as pd
import numpy as np
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import MF, BPR, NeuMF, VAECF, LightGCN
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
from cornac.experiment import Experiment

df = pd.read_csv('hotels_users_ratings.csv')
data = [(f"{row['UserID']}", f"{row['HotelID']}", row['Rating']) for _, row in df.iterrows()]


In [10]:
data

[('7342', '2638', 9),
 ('5732', '2638', 10),
 ('9445', '2638', 10),
 ('5508', '2638', 10),
 ('7770', '2638', 10),
 ('3988', '2638', 10),
 ('1289', '1456', 2),
 ('4001', '1456', 2),
 ('2902', '1456', 1),
 ('2037', '1456', 1),
 ('884', '1456', 8),
 ('997', '1456', 7),
 ('507', '1456', 8),
 ('7179', '1456', 9),
 ('5204', '1456', 8),
 ('9076', '1456', 10),
 ('10353', '1456', 3),
 ('7174', '1456', 1),
 ('10839', '1456', 4),
 ('8355', '1456', 5),
 ('11021', '1456', 5),
 ('5529', '1456', 6),
 ('9359', '1456', 1),
 ('840', '1456', 10),
 ('3298', '1456', 10),
 ('9470', '1456', 10),
 ('3059', '1064', 5),
 ('9470', '1064', 1),
 ('273', '1064', 7),
 ('6916', '1064', 7),
 ('9463', '1064', 5),
 ('10097', '1064', 7),
 ('8092', '1064', 6),
 ('7221', '1064', 3),
 ('4506', '1064', 4),
 ('9352', '1064', 5),
 ('2882', '2225', 10),
 ('10458', '2225', 9),
 ('4246', '2225', 10),
 ('9726', '2225', 9),
 ('1951', '2225', 10),
 ('11042', '2225', 10),
 ('6654', '2225', 8),
 ('9194', '2225', 10),
 ('8993', '2225',

In [11]:
rs = RatioSplit(data, 
                test_size=0.2,
                rating_threshold=1.0,
                seed=123,
                verbose=True)


mf = MF(k=10, 
        max_iter=100, 
        learning_rate=0.01, 
        seed=123,
        verbose=True)

bpr = BPR(k=10, 
          max_iter=100, 
          learning_rate=0.01, 
          seed=123,
          verbose=True)

vaecf = VAECF(k=10,
              autoencoder_structure=[20],
              act_fn="tanh",
              likelihood="pois",
              n_epochs=100,
              batch_size=2048,
              seed=123,
              verbose=True)

lightgcn = LightGCN(
            seed=123,
            num_epochs=100,
            num_layers=3,
            early_stopping={"min_delta": 1e-4, "patience": 50},
            batch_size=2048,
            learning_rate=0.001,
            lambda_reg=1e-4,
            verbose=True
        )


metrics = [Precision(k=10), Recall(k=10), NDCG(k=10), AUC(), MAP()]


exp = Experiment(eval_method=rs,
                models=[mf, bpr, vaecf, lightgcn],
                metrics=metrics,
                user_based=True,
                verbose=True)
exp.run()

c:\Users\Administrator\Downloads\DS307\venv\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 8140 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
c:\Users\Administrator\Downloads\DS307\venv\lib\site-packages\cornac\data\dataset.py:335: UserWarning: 698 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 9680
Number of items = 4446
Number of ratings = 38900
Max rating = 10.0
Min rating = 1.0
Global mean = 7.8
---
Test data:
Number of users = 9680
Number of items = 4446
Number of ratings = 9406
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 9680
Total items = 4446

[MF] Training started!


100%|██████████| 100/100 [00:00<00:00, 829.21it/s, loss=1838.31]


Optimization finished!

[MF] Evaluation started!


Ranking: 100%|██████████| 2384/2384 [00:02<00:00, 1015.70it/s]


...


[LightGCN] Training started!


Training:   0%|          | 0/100 [00:00<?, ?iter/s]c:\Users\Administrator\Downloads\DS307\venv\lib\site-packages\dgl\backend\pytorch\tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (



[LightGCN] Evaluation started!


Ranking: 100%|██████████| 2384/2384 [00:02<00:00, 1080.54it/s]


TEST:
...
         |    AUC |    MAP | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
-------- + ------ + ------ + ------- + ------------ + --------- + --------- + --------
MF       | 0.6554 | 0.0284 |  0.0367 |       0.0135 |    0.0735 |    0.1261 |   2.3501
BPR      | 0.7784 | 0.1526 |  0.2235 |       0.0891 |    0.3185 |    0.4301 |   2.4129
VAECF    | 0.7078 | 0.1538 |  0.2231 |       0.0859 |    0.3188 |  163.9940 |   2.8109
LightGCN | 0.8327 | 0.2145 |  0.2865 |       0.0994 |    0.3684 |  175.5239 |   2.2083



In [12]:
lightgcn.save("./venv/var/save_dir", save_trainset=True)

LightGCN model is saved to ./venv/var/save_dir\LightGCN\2024-12-15_17-38-10-302739.pkl


'./venv/var/save_dir\\LightGCN\\2024-12-15_17-38-10-302739.pkl'

In [14]:
lightgcn.recommend(k=2, user_id='1')

['2841', '2225']